In [10]:
import re
import sys
import itertools as it
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

In [11]:
with open('browsing.txt', 'r') as browsing:
	lines = browsing.readlines()

with open('browsingupdated.txt', 'w') as bbrowsing:
	for (num, line) in enumerate(lines):
		bbrowsing.write('%d:%s' % (num, line))

lines = sc.textFile('browsingupdated.txt')
items = lines.map(lambda l: l.split(':'))
finalitems = items.map(lambda p: (int(p[0]), sorted(p[1].split())))

In [12]:
s = 100

In [13]:
def singleitems(basket):
	basketId = basket[0]
	itemList = basket[1]
	return [(item, 1) for item in itemList]

In [14]:
singles = finalitems.map(singleitems).flatMap(lambda x: x)
itemCounts = singles.reduceByKey(lambda c1, c2: c1 + c2)
frequentitems = itemCounts.filter(lambda x: x[1] >= s)
freq1 = frequentitems.collectAsMap()

In [15]:
def frequentSingles(basket):
	basketId = basket[0]
	itemList = basket[1]
	freqList = []
	for item in itemList:
		if item in freq1:
			freqList.append(item)
	return (basketId, sorted(freqList))

In [16]:
freqSingles = finalitems.map(frequentSingles)

In [17]:
def itempairs(basket):
	basketId = basket[0]
	itemList = basket[1]
	return [(pairs, 1) for pairs in it.combinations(itemList, 2)]

In [18]:
doubles = freqSingles.map(itempairs).flatMap(lambda x: x)
doubleCounts = doubles.reduceByKey(lambda c1, c2: c1 + c2)
frequentpairs = doubleCounts.filter(lambda x: x[1] >= s)
freq2 = frequentpairs.collectAsMap()

In [19]:
def frequentDoubles(basket_freqSingles):
	basketId = basket_freqSingles[0]
	frequent = basket_freqSingles[1]
	pairList = []
	CandList = [pairs for pairs in it.combinations(frequent, 2)]
	for pair in CandList:
		if pair in freq2:
			pairList.append(pair)
	return (basketId, sorted(pairList))

In [20]:
freqDoubles = freqSingles.map(frequentDoubles)

In [21]:
def itemtriplets(freqPairs):
	basketId   = freqPairs[0]
	freqpairs = freqPairs[1]
	freqitems = []
	for p in freqpairs:
		i1, i2 = p
		if i1 not in freqitems:
			freqitems.append(i1)
		if i2 not in freqitems:
			freqitems.append(i2)
	return [(triples, 1) for triples in it.combinations(sorted(freqitems), 3)]

In [22]:
triples = freqDoubles.map(itemtriplets).flatMap(lambda x: x)
tripleCounts = triples.reduceByKey(lambda c1, c2: c1 + c2)
frequenttriplets = tripleCounts.filter(lambda x: x[1] >= s)
freq3 = frequenttriplets.collectAsMap()

In [23]:
def pairssortbyConf(pairs, items):
	ruleList = []
	for pair in pairs:
		x = pair[0]
		y = pair[1]
		X_support = items[x]
		Y_support = items[y]
		XYsupport = pairs[pair]
		Conf_XtoY = (1.0 * XYsupport)/X_support
		Conf_YtoX = (1.0 * XYsupport)/Y_support
		ruleList.append((Conf_XtoY, (x, y)))
		ruleList.append((Conf_YtoX, (y, x)))
	return sorted(ruleList, key = lambda x: (-x[0], x[1]))

In [24]:
def tripletssortByConf(triples, pairs):
	ruleList = []
	for triple in triples:
		x = triple[0]
		y = triple[1]
		z = triple[2]
		XY_support = pairs[(x, y)]
		XZ_support = pairs[(x, z)]
		YZ_support = pairs[(y, z)]
		XYZsupport = triples[triple]
		Conf_XYtoZ = (1.0 * XYZsupport)/XY_support
		Conf_XZtoY = (1.0 * XYZsupport)/XZ_support
		Conf_YZtoX = (1.0 * XYZsupport)/YZ_support
		ruleList.append((Conf_XYtoZ, ((x, y), z)))
		ruleList.append((Conf_XZtoY, ((x, z), y)))
		ruleList.append((Conf_YZtoX, ((y, z), x)))
	return sorted(ruleList, key = lambda x: (-x[0], x[1]))

In [25]:
one2oneRules = pairssortbyConf(freq2, freq1)
for i in range(5):
	print("%s => %s : %.8f" % (one2oneRules[i][1][0], one2oneRules[i][1][1], one2oneRules[i][0]))

print("\n")

two2oneRules = tripletssortByConf(freq3, freq2)
for i in range(5):
	print("(%s, %s) => %s : %.8f" % (two2oneRules[i][1][0][0], two2oneRules[i][1][0][1], two2oneRules[i][1][1], two2oneRules[i][0]))

DAI93865 => FRO40251 : 1.00000000
GRO85051 => FRO40251 : 0.99917628
GRO38636 => FRO40251 : 0.99065421
ELE12951 => FRO40251 : 0.99056604
DAI88079 => FRO40251 : 0.98672566


(DAI23334, ELE92920) => DAI62779 : 1.00000000
(DAI31081, GRO85051) => FRO40251 : 1.00000000
(DAI55911, GRO85051) => FRO40251 : 1.00000000
(DAI62779, DAI88079) => FRO40251 : 1.00000000
(DAI75645, GRO85051) => FRO40251 : 1.00000000


In [26]:
sc.stop()